In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, TensorDataset
from transformers import BertModel, BertTokenizer, BertConfig
from sklearn.preprocessing import StandardScaler
import torch.optim as optim

In [ ]:
# Load the SECOM dataset
secom_data_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/secom/secom.data'
secom_labels_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/secom/secom_labels.data'

# Load features and labels
secom_df = pd.read_csv(secom_data_url, sep=" ", header=None)
secom_labels = pd.read_csv(secom_labels_url, sep=" ", header=None)

# Preprocessing: handle missing values and scale data
# Replace 'NaN' values with the mean of the respective columns
secom_df.replace('NaN', np.nan, inplace=True)
secom_df = secom_df.fillna(secom_df.mean())

# Normalize data
scaler = StandardScaler()
secom_df = scaler.fit_transform(secom_df)

# Convert labels to binary
secom_labels.columns = ['Result', 'Timestamp']
secom_labels['Result'] = secom_labels['Result'].apply(lambda x: 1 if x == -1 else 0)

In [ ]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(secom_df, secom_labels['Result'], test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

In [ ]:
# Define a custom dataset
class SECOMDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

train_dataset = SECOMDataset(X_train_tensor, y_train_tensor)
test_dataset = SECOMDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [ ]:
# Define Transformer Model for classification
class TransformerModel(nn.Module):
    def __init__(self, input_dim, num_classes, hidden_dim=256, nhead=8, num_layers=3):
        super(TransformerModel, self).__init__()
        self.embedding = nn.Linear(input_dim, hidden_dim)
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=hidden_dim, nhead=nhead),
            num_layers=num_layers
        )
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        x = self.embedding(x)
        x = x.unsqueeze(0)  # Add batch dimension for transformer input
        x = self.transformer(x)
        x = x.squeeze(0)  # Remove batch dimension
        x = self.fc(x)
        return x

# Model hyperparameters
input_dim = X_train.shape[1]  # 591 features
num_classes = 2  # Binary classification
hidden_dim = 256
nhead = 8
num_layers = 3

# Initialize model, loss function, and optimizer
model = TransformerModel(input_dim, num_classes, hidden_dim, nhead, num_layers)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Function to train the model
def train_model(model, train_loader, criterion, optimizer, epochs=10):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f'Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader)}')

# Function to evaluate the model
def evaluate_model(model, test_loader):
    model.eval()
    y_pred = []
    y_true = []
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            y_pred.extend(predicted.cpu().numpy())
            y_true.extend(labels.cpu().numpy())
    return y_true, y_pred

# Train the model
train_model(model, train_loader, criterion, optimizer, epochs=10)

# Evaluate the model
y_true, y_pred = evaluate_model(model, test_loader)

# Print accuracy and classification report
print(f"Accuracy: {accuracy_score(y_true, y_pred)}")
print(classification_report(y_true, y_pred))


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch 1/10, Loss: 0.28797928085550667
Epoch 2/10, Loss: 0.22779787317849695
Epoch 3/10, Loss: 0.2314736731350422
Epoch 4/10, Loss: 0.23654514928348364
Epoch 5/10, Loss: 0.23307902477681636
Epoch 6/10, Loss: 0.2402514386922121
Epoch 7/10, Loss: 0.24077847562730312
Epoch 8/10, Loss: 0.24196320306509733
Epoch 9/10, Loss: 0.2369678294286132
Epoch 10/10, Loss: 0.24082234762609006
Accuracy: 0.9235668789808917
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        24
           1       0.92      1.00      0.96       290

    accuracy                           0.92       314
   macro avg       0.46      0.50      0.48       314
weighted avg       0.85      0.92      0.89       314



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Add dropout to the transformer model to prevent overfitting.
Use KNN imputation for missing values.
Use PCA for feature reduction.

In [ ]:
from sklearn.impute import KNNImputer
from sklearn.decomposition import PCA

# Step 1: KNN Imputation for missing values
imputer = KNNImputer(n_neighbors=5)  # Use 5 nearest neighbors for imputation
secom_df_imputed = imputer.fit_transform(secom_df)

# Step 2: PCA for feature reduction
pca = PCA(n_components=100)  # Reduce to 100 components for now
secom_df_reduced = pca.fit_transform(secom_df_imputed)

# Update train-test split with reduced features
X_train, X_test, y_train, y_test = train_test_split(secom_df_reduced, secom_labels['Result'], test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

# Custom dataset and data loaders remain the same
train_dataset = SECOMDataset(X_train_tensor, y_train_tensor)
test_dataset = SECOMDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Updated Transformer model with Dropout
class TransformerModel(nn.Module):
    def __init__(self, input_dim, num_classes, hidden_dim=256, nhead=8, num_layers=3, dropout=0.3):
        super(TransformerModel, self).__init__()
        self.embedding = nn.Linear(input_dim, hidden_dim)
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=hidden_dim, nhead=nhead, dropout=dropout),
            num_layers=num_layers
        )
        self.fc = nn.Linear(hidden_dim, num_classes)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.embedding(x)
        x = self.dropout(x)  # Apply dropout after embedding
        x = x.unsqueeze(0)  # Add batch dimension for transformer input
        x = self.transformer(x)
        x = x.squeeze(0)  # Remove batch dimension
        x = self.fc(x)
        return x

# Initialize model with Dropout, optimizer, and loss function
model = TransformerModel(input_dim=X_train.shape[1], num_classes=2, hidden_dim=256, nhead=8, num_layers=3, dropout=0.3)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train and evaluate the model
train_model(model, train_loader, criterion, optimizer, epochs=10)
y_true, y_pred = evaluate_model(model, test_loader)

# Print accuracy and classification report
print(f"Accuracy: {accuracy_score(y_true, y_pred)}")
print(classification_report(y_true, y_pred))


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch 1/10, Loss: 0.3143065898140776
Epoch 2/10, Loss: 0.2420719064772129
Epoch 3/10, Loss: 0.2366992337629199
Epoch 4/10, Loss: 0.2417006280273199
Epoch 5/10, Loss: 0.23763134167529643
Epoch 6/10, Loss: 0.24318651538342237
Epoch 7/10, Loss: 0.23186061913147568
Epoch 8/10, Loss: 0.24765461636707187
Epoch 9/10, Loss: 0.25093495175242425
Epoch 10/10, Loss: 0.23716117814183235
Accuracy: 0.9235668789808917
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        24
           1       0.92      1.00      0.96       290

    accuracy                           0.92       314
   macro avg       0.46      0.50      0.48       314
weighted avg       0.85      0.92      0.89       314



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Updated Code with SMOTE and Learning Rate Scheduler

In [ ]:
from imblearn.over_sampling import SMOTE
import torch.optim.lr_scheduler as lr_scheduler

# Step 1: Apply SMOTE for class balancing
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(secom_df_reduced, secom_labels['Result'])

# Step 2: Train-test split with balanced data
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Convert to PyTorch tensors for both features and labels
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)

# Convert the labels to NumPy arrays before creating tensors
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)  # Use .values to convert Series to NumPy array
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)    # Same for y_test

# Create dataset and dataloaders (same as before)
train_dataset = SECOMDataset(X_train_tensor, y_train_tensor)
test_dataset = SECOMDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


# Define model (same as previous with Dropout)
class TransformerModel(nn.Module):
    def __init__(self, input_dim, num_classes, hidden_dim=256, nhead=8, num_layers=3, dropout=0.3):
        super(TransformerModel, self).__init__()
        self.embedding = nn.Linear(input_dim, hidden_dim)
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=hidden_dim, nhead=nhead, dropout=dropout),
            num_layers=num_layers
        )
        self.fc = nn.Linear(hidden_dim, num_classes)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.embedding(x)
        x = self.dropout(x)  # Apply dropout after embedding
        x = x.unsqueeze(0)  # Add batch dimension for transformer input
        x = self.transformer(x)
        x = x.squeeze(0)  # Remove batch dimension
        x = self.fc(x)
        return x

# Initialize model, criterion, optimizer, and scheduler
model = TransformerModel(input_dim=X_train.shape[1], num_classes=2, hidden_dim=256, nhead=8, num_layers=3, dropout=0.3)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Step 3: Use a learning rate scheduler to reduce the learning rate when the loss plateaus
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)

# Training function with scheduler
def train_model_with_scheduler(model, train_loader, criterion, optimizer, scheduler, epochs=10):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        # Step the scheduler after each epoch based on the running loss
        scheduler.step(running_loss / len(train_loader))
        print(f'Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader)}')

# Train the model
train_model_with_scheduler(model, train_loader, criterion, optimizer, scheduler, epochs=10)

# Evaluate the model
y_true, y_pred = evaluate_model(model, test_loader)

# Print accuracy and classification report
print(f"Accuracy: {accuracy_score(y_true, y_pred)}")
print(classification_report(y_true, y_pred))


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch 1/10, Loss: 0.5643275331806492
Epoch 2/10, Loss: 0.4232871512303481
Epoch 3/10, Loss: 0.30302662417493964
Epoch 4/10, Loss: 0.23682577042160807
Epoch 5/10, Loss: 0.1983211965274972
Epoch 6/10, Loss: 0.21706474761201724
Epoch 7/10, Loss: 0.19205411775289355
Epoch 8/10, Loss: 0.1389718122176222
Epoch 9/10, Loss: 0.13289408778419365
Epoch 10/10, Loss: 0.13806186978881424
Accuracy: 0.9505119453924915
              precision    recall  f1-score   support

           0       0.91      0.99      0.95       280
           1       0.99      0.92      0.95       306

    accuracy                           0.95       586
   macro avg       0.95      0.95      0.95       586
weighted avg       0.95      0.95      0.95       586



Implementing PCA for Further Feature Reduction

In [ ]:
from sklearn.decomposition import PCA

# Step 1: Apply PCA for dimensionality reduction
pca = PCA(n_components=100)  # Reducing to 100 components
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Convert to PyTorch tensors
X_train_tensor_pca = torch.tensor(X_train_pca, dtype=torch.float32)
X_test_tensor_pca = torch.tensor(X_test_pca, dtype=torch.float32)

# Keep labels the same
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

# Custom dataset and dataloaders for PCA reduced data
train_dataset_pca = SECOMDataset(X_train_tensor_pca, y_train_tensor)
test_dataset_pca = SECOMDataset(X_test_tensor_pca, y_test_tensor)

train_loader_pca = DataLoader(train_dataset_pca, batch_size=32, shuffle=True)
test_loader_pca = DataLoader(test_dataset_pca, batch_size=32, shuffle=False)

# Training the model (same as before)
train_model_with_scheduler(model, train_loader_pca, criterion, optimizer, scheduler, epochs=10)

# Evaluate the model
y_true_pca, y_pred_pca = evaluate_model(model, test_loader_pca)

# Print accuracy and classification report
print(f"Accuracy after PCA: {accuracy_score(y_true_pca, y_pred_pca)}")
print(classification_report(y_true_pca, y_pred_pca))


Epoch 1/10, Loss: 0.732347804146844
Epoch 2/10, Loss: 0.6309620476252323
Epoch 3/10, Loss: 0.5462336113324037
Epoch 4/10, Loss: 0.4713553133848551
Epoch 5/10, Loss: 0.4364694417328448
Epoch 6/10, Loss: 0.4287860329086716
Epoch 7/10, Loss: 0.4053628968225943
Epoch 8/10, Loss: 0.3980459983687143
Epoch 9/10, Loss: 0.3830716400533109
Epoch 10/10, Loss: 0.3795709873776178
Accuracy after PCA: 0.8344709897610921
              precision    recall  f1-score   support

           0       0.84      0.80      0.82       280
           1       0.83      0.86      0.84       306

    accuracy                           0.83       586
   macro avg       0.84      0.83      0.83       586
weighted avg       0.83      0.83      0.83       586



In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

# Load the SECOM dataset
secom_data_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/secom/secom.data'
secom_labels_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/secom/secom_labels.data'

# Load features and labels
secom_df = pd.read_csv(secom_data_url, sep=" ", header=None)
secom_labels = pd.read_csv(secom_labels_url, sep=" ", header=None)

# Preprocessing: handle missing values and scale data
secom_df.replace('NaN', np.nan, inplace=True)
imputer = KNNImputer(n_neighbors=5)
secom_df_imputed = imputer.fit_transform(secom_df)

# Normalize data
scaler = StandardScaler()
secom_df_scaled = scaler.fit_transform(secom_df_imputed)

# Convert labels to binary
secom_labels.columns = ['Result', 'Timestamp']
secom_labels['Result'] = secom_labels['Result'].apply(lambda x: 1 if x == -1 else 0)

# Step 1: Apply PCA for feature reduction
pca = PCA(n_components=100)  # Reduce to 100 components for now
secom_df_reduced = pca.fit_transform(secom_df_scaled)

# Step 2: Apply SMOTE for class balancing
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(secom_df_reduced, secom_labels['Result'])

# Train-test split with balanced data
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Convert to PyTorch tensors for both features and labels
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

# Define a custom dataset
class SECOMDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

train_dataset = SECOMDataset(X_train_tensor, y_train_tensor)
test_dataset = SECOMDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Define Transformer Model for classification with Dropout
class TransformerModel(nn.Module):
    def __init__(self, input_dim, num_classes, hidden_dim=256, nhead=8, num_layers=3, dropout=0.3):
        super(TransformerModel, self).__init__()
        self.embedding = nn.Linear(input_dim, hidden_dim)
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=hidden_dim, nhead=nhead, dropout=dropout),
            num_layers=num_layers)
        self.fc = nn.Linear(hidden_dim, num_classes)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.embedding(x)
        x = self.dropout(x)  # Apply dropout after embedding
        x = x.unsqueeze(0)  # Add batch dimension for transformer input
        x = self.transformer(x)
        x = x.squeeze(0)  # Remove batch dimension
        x = self.fc(x)
        return x

# Initialize model, criterion, optimizer and scheduler
model = TransformerModel(input_dim=X_train.shape[1], num_classes=2)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3)

# Function to train the model with scheduler
def train_model_with_scheduler(model, train_loader, criterion, optimizer, scheduler, epochs=10):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        # Step the scheduler after each epoch based on the running loss
        scheduler.step(running_loss / len(train_loader))
        print(f'Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader)}')

# Function to evaluate the model
def evaluate_model(model, test_loader):
    model.eval()
    y_pred = []
    y_true = []
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            y_pred.extend(predicted.cpu().numpy())
            y_true.extend(labels.cpu().numpy())
    return y_true, y_pred

# Train the model
train_model_with_scheduler(model, train_loader, criterion, optimizer, scheduler, epochs=10)

# Evaluate the model
y_true, y_pred = evaluate_model(model, test_loader)

# Print accuracy and classification report
print(f"Accuracy: {accuracy_score(y_true, y_pred)}")
print(classification_report(y_true, y_pred))

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


Epoch 1/10, Loss: 0.5647906240679927
Epoch 2/10, Loss: 0.4266208688149581
Epoch 3/10, Loss: 0.329221808427089
Epoch 4/10, Loss: 0.31718335285581445
Epoch 5/10, Loss: 0.23962807141848513
Epoch 6/10, Loss: 0.19725953656676654
Epoch 7/10, Loss: 0.17302653559710127
Epoch 8/10, Loss: 0.15802848477520653
Epoch 9/10, Loss: 0.1377283816439779
Epoch 10/10, Loss: 0.10893618119125431
Accuracy: 0.9436860068259386
              precision    recall  f1-score   support

           0       0.91      0.98      0.94       280
           1       0.98      0.91      0.94       306

    accuracy                           0.94       586
   macro avg       0.94      0.95      0.94       586
weighted avg       0.95      0.94      0.94       586



In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

# Load the SECOM dataset
secom_data_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/secom/secom.data'
secom_labels_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/secom/secom_labels.data'

# Load features and labels
secom_df = pd.read_csv(secom_data_url, sep=" ", header=None)
secom_labels = pd.read_csv(secom_labels_url, sep=" ", header=None)

# Preprocessing: handle missing values and scale data
imputer = KNNImputer(n_neighbors=5)
secom_df_imputed = imputer.fit_transform(secom_df)
scaler = StandardScaler()
secom_df_scaled = scaler.fit_transform(secom_df_imputed)

# Convert labels to binary
secom_labels.columns = ['Result', 'Timestamp']
secom_labels['Result'] = secom_labels['Result'].apply(lambda x: 1 if x == -1 else 0)

# Step 1: Apply PCA for feature reduction
pca = PCA(n_components=100)
secom_df_reduced = pca.fit_transform(secom_df_scaled)

# Step 2: Apply SMOTE for class balancing
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(secom_df_reduced, secom_labels['Result'])

# Train-test split with balanced data
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Convert to PyTorch tensors for both features and labels
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

# Define a custom dataset
class SECOMDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

train_dataset = SECOMDataset(X_train_tensor, y_train_tensor)
test_dataset = SECOMDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Define Transformer Model for classification with Dropout
class TransformerModel(nn.Module):
    def __init__(self, input_dim, num_classes, hidden_dim=256, nhead=8, num_layers=3, dropout=0.3):
        super(TransformerModel, self).__init__()
        self.embedding = nn.Linear(input_dim, hidden_dim)
        self.transformer_layers = nn.ModuleList([
            nn.TransformerEncoderLayer(d_model=hidden_dim, nhead=nhead, dropout=dropout) for _ in range(num_layers)
        ])
        self.fc = nn.Linear(hidden_dim, num_classes)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.embedding(x)  # Shape: (batch_size, hidden_dim)
        x = self.dropout(x)    # Apply dropout after embedding

        # Reshape for transformer: (batch_size, seq_len (1), hidden_dim)
        x = x.unsqueeze(1)      # Add sequence length dimension

        for layer in self.transformer_layers:
            x = layer(x)         # Shape: (batch_size, seq_len (1), hidden_dim)

        x = x.squeeze(1)         # Remove sequence length dimension
        x = self.fc(x)           # Shape: (batch_size, num_classes)

        return x

# Initialize model with Dropout and optimizer
model = TransformerModel(input_dim=X_train.shape[1], num_classes=2)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Learning rate scheduler setup
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3)

# Function to train the model with scheduler
def train_model_with_scheduler(model, train_loader, criterion, optimizer, scheduler, epochs=20):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        # Step the scheduler after each epoch based on the running loss
        scheduler.step(running_loss / len(train_loader))
        print(f'Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader)}')

# Function to evaluate the model
def evaluate_model(model, test_loader):
    model.eval()
    y_pred = []
    y_true = []
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            y_pred.extend(predicted.cpu().numpy())
            y_true.extend(labels.cpu().numpy())
    return y_true, y_pred

# Train the model with increased epochs and improved structure
train_model_with_scheduler(model, train_loader, criterion, optimizer, scheduler, epochs=20)

# Evaluate the model and print results
y_true, y_pred = evaluate_model(model, test_loader)
print(f"Accuracy: {accuracy_score(y_true, y_pred)}")
print(classification_report(y_true, y_pred))

Epoch 1/20, Loss: 0.7831966208445059
Epoch 2/20, Loss: 0.5095763150099162
Epoch 3/20, Loss: 0.4603849321201041
Epoch 4/20, Loss: 0.42416659057945816
Epoch 5/20, Loss: 0.38424061601226395
Epoch 6/20, Loss: 0.34833383182617456
Epoch 7/20, Loss: 0.27861903310828917
Epoch 8/20, Loss: 0.2965499191872171
Epoch 9/20, Loss: 0.28866263176943807
Epoch 10/20, Loss: 0.2989032208114057
Epoch 11/20, Loss: 0.2449187186320086
Epoch 12/20, Loss: 0.2861582774266198
Epoch 13/20, Loss: 0.2734248718699893
Epoch 14/20, Loss: 0.27398209726891004
Epoch 15/20, Loss: 0.3091446297192896
Epoch 16/20, Loss: 0.324730424260771
Epoch 17/20, Loss: 0.24699894157615868
Epoch 18/20, Loss: 0.21663895888707121
Epoch 19/20, Loss: 0.2076412876312797
Epoch 20/20, Loss: 0.18402271133822365
Accuracy: 0.9505119453924915
              precision    recall  f1-score   support

           0       0.93      0.97      0.95       280
           1       0.98      0.93      0.95       306

    accuracy                           0.95     

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

# Load the SECOM dataset
secom_data_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/secom/secom.data'
secom_labels_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/secom/secom_labels.data'

# Load features and labels
secom_df = pd.read_csv(secom_data_url, sep=" ", header=None)
secom_labels = pd.read_csv(secom_labels_url, sep=" ", header=None)

# Preprocessing: handle missing values and scale data
imputer = KNNImputer(n_neighbors=5)
secom_df_imputed = imputer.fit_transform(secom_df)
scaler = StandardScaler()
secom_df_scaled = scaler.fit_transform(secom_df_imputed)

# Convert labels to binary
secom_labels.columns = ['Result', 'Timestamp']
secom_labels['Result'] = secom_labels['Result'].apply(lambda x: 1 if x == -1 else 0)

# Apply SMOTE for class balancing
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(secom_df_scaled, secom_labels['Result'])

# Train-test split with balanced data
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Convert to PyTorch tensors for both features and labels
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

# Define a custom dataset
class SECOMDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

train_dataset = SECOMDataset(X_train_tensor, y_train_tensor)
test_dataset = SECOMDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Define Transformer Model for classification with Dropout and Batch Normalization
class TransformerModel(nn.Module):
    def __init__(self, input_dim, num_classes, hidden_dim=256, nhead=8, num_layers=3, dropout=0.3):
        super(TransformerModel, self).__init__()
        self.embedding = nn.Linear(input_dim, hidden_dim)
        self.transformer_layers = nn.ModuleList([
            nn.TransformerEncoderLayer(d_model=hidden_dim, nhead=nhead, dropout=dropout) for _ in range(num_layers)
        ])
        self.fc = nn.Linear(hidden_dim, num_classes)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.embedding(x)  # Shape: (batch_size, hidden_dim)
        x = self.dropout(x)    # Apply dropout after embedding

        # Reshape for transformer: (batch_size, seq_len (1), hidden_dim)
        x = x.unsqueeze(1)      # Add sequence length dimension

        for layer in self.transformer_layers:
            x = layer(x)         # Shape: (batch_size, seq_len (1), hidden_dim)

        x = x.squeeze(1)         # Remove sequence length dimension
        x = self.fc(x)           # Shape: (batch_size, num_classes)

        return x

# Initialize model with Dropout and optimizer
model = TransformerModel(input_dim=X_train.shape[1], num_classes=2)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Learning rate scheduler setup
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3)

# Function to train the model with scheduler
def train_model_with_scheduler(model, train_loader, criterion, optimizer, scheduler, epochs=20):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        # Step the scheduler after each epoch based on the running loss
        scheduler.step(running_loss / len(train_loader))
        print(f'Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader)}')

# Function to evaluate the model
def evaluate_model(model, test_loader):
    model.eval()
    y_pred = []
    y_true = []
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            y_pred.extend(predicted.cpu().numpy())
            y_true.extend(labels.cpu().numpy())
    return y_true, y_pred

# Train the model with increased epochs and improved structure
train_model_with_scheduler(model, train_loader, criterion, optimizer, scheduler, epochs=20)

# Evaluate the model and print results
y_true, y_pred = evaluate_model(model, test_loader)
print(f"Accuracy: {accuracy_score(y_true, y_pred)}")
print(classification_report(y_true, y_pred))

Epoch 1/20, Loss: 0.49915729422827027
Epoch 2/20, Loss: 0.2744014007920349
Epoch 3/20, Loss: 0.2578466400202062
Epoch 4/20, Loss: 0.3891297385499284
Epoch 5/20, Loss: 0.29761497062203046
Epoch 6/20, Loss: 0.38679545231767604
Epoch 7/20, Loss: 0.32348049683748065
Epoch 8/20, Loss: 0.3150554259081145
Epoch 9/20, Loss: 0.2743542015149787
Epoch 10/20, Loss: 0.23878937350535714
Epoch 11/20, Loss: 0.22856573508800687
Epoch 12/20, Loss: 0.21227628304748922
Epoch 13/20, Loss: 0.18278607917395798
Epoch 14/20, Loss: 0.17113674119920344
Epoch 15/20, Loss: 0.16065678433389277
Epoch 16/20, Loss: 0.15169508299614126
Epoch 17/20, Loss: 0.13662615712933443
Epoch 18/20, Loss: 0.12366162214075795
Epoch 19/20, Loss: 0.13835279768131473
Epoch 20/20, Loss: 0.10786745460653627
Accuracy: 0.9419795221843004
              precision    recall  f1-score   support

           0       0.89      1.00      0.94       280
           1       1.00      0.89      0.94       306

    accuracy                           0.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from sklearn.decomposition import PCA
from sklearn.impute import KNNImputer

# Load the SECOM dataset
secom_data_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/secom/secom.data'
secom_labels_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/secom/secom_labels.data'

# Load features and labels
secom_df = pd.read_csv(secom_data_url, sep=" ", header=None)
secom_labels = pd.read_csv(secom_labels_url, sep=" ", header=None)

# Preprocessing: handle missing values and scale data
secom_df.replace('NaN', np.nan, inplace=True)
imputer = KNNImputer(n_neighbors=5)
secom_df_imputed = imputer.fit_transform(secom_df)

# Normalize data
scaler = StandardScaler()
secom_df_scaled = scaler.fit_transform(secom_df_imputed)

# Convert labels to binary
secom_labels.columns = ['Result', 'Timestamp']
secom_labels['Result'] = secom_labels['Result'].apply(lambda x: 1 if x == -1 else 0)

# Step 1: Apply SMOTE for class balancing
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(secom_df_scaled, secom_labels['Result'])

# Step 2: Train-test split with balanced data
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Convert to PyTorch tensors for both features and labels
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

# Define a custom dataset
class SECOMDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

train_dataset = SECOMDataset(X_train_tensor, y_train_tensor)
test_dataset = SECOMDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Define Transformer Model for classification with Dropout and Batch Normalization
class TransformerModel(nn.Module):
    def __init__(self, input_dim, num_classes, hidden_dim=256, nhead=8, num_layers=3, dropout=0.3):
        super(TransformerModel, self).__init__()
        self.embedding = nn.Linear(input_dim, hidden_dim)
        self.transformer_layers = nn.ModuleList([
            nn.TransformerEncoderLayer(d_model=hidden_dim, nhead=nhead, dropout=dropout) for _ in range(num_layers)
        ])
        self.fc = nn.Linear(hidden_dim, num_classes)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.embedding(x)  # Shape: (batch_size, hidden_dim)
        x = self.dropout(x)    # Apply dropout after embedding

        # Reshape for transformer: (batch_size, seq_len (1), hidden_dim)
        x = x.unsqueeze(1)      # Add sequence length dimension

        for layer in self.transformer_layers:
            x = layer(x)         # Shape: (batch_size, seq_len (1), hidden_dim)

        x = x.squeeze(1)         # Remove sequence length dimension
        x = self.fc(x)           # Shape: (batch_size, num_classes)

        return x

# Initialize model with Dropout and optimizer
model = TransformerModel(input_dim=X_train.shape[1], num_classes=2)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Learning rate scheduler setup
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3)

# Function to train the model with scheduler
def train_model_with_scheduler(model, train_loader, criterion, optimizer, scheduler, epochs=20):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        # Step the scheduler after each epoch based on the running loss
        scheduler.step(running_loss / len(train_loader))
        print(f'Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader)}')

# Function to evaluate the model
def evaluate_model(model, test_loader):
    model.eval()
    y_pred = []
    y_true = []
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            y_pred.extend(predicted.cpu().numpy())
            y_true.extend(labels.cpu().numpy())
    return y_true, y_pred

# Train the model with increased epochs and improved structure
train_model_with_scheduler(model, train_loader, criterion, optimizer, scheduler, epochs=20)

# Evaluate the model and print results
y_true_pca , y_pred_pca  = evaluate_model(model,test_loader)
print(f"Accuracy after PCA: {accuracy_score(y_true_pca , y_pred_pca )}")
print(classification_report(y_true_pca , y_pred_pca ))

Epoch 1/20, Loss: 0.5021539733216569
Epoch 2/20, Loss: 0.28203576707558053
Epoch 3/20, Loss: 0.2593970114012828
Epoch 4/20, Loss: 0.2455261353201963
Epoch 5/20, Loss: 0.2585328446348777
Epoch 6/20, Loss: 0.26943880709743984
Epoch 7/20, Loss: 0.32298274642813046
Epoch 8/20, Loss: 0.30910241966311996
Epoch 9/20, Loss: 0.22280928552956195
Epoch 10/20, Loss: 0.19848586336986437
Epoch 11/20, Loss: 0.16284939108064045
Epoch 12/20, Loss: 0.1596764937244557
Epoch 13/20, Loss: 0.14716129203805248
Epoch 14/20, Loss: 0.12157320224906544
Epoch 15/20, Loss: 0.12290389744590062
Epoch 16/20, Loss: 0.09367998700739967
Epoch 17/20, Loss: 0.09350105815541905
Epoch 18/20, Loss: 0.09122443158884307
Epoch 19/20, Loss: 0.08471216303548096
Epoch 20/20, Loss: 0.07160097521075325
Accuracy after PCA: 0.9453924914675768
              precision    recall  f1-score   support

           0       0.90      1.00      0.95       280
           1       1.00      0.90      0.95       306

    accuracy                   